<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/RF(GWAS%2C3000).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2016년 표현형 데이터.xlsx to 2016년 표현형 데이터.xlsx
Saving GWAS_SNP(3000).csv to GWAS_SNP(3000).csv
Saving 유전형인코딩_최빈값.csv to 유전형인코딩_최빈값.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
import warnings
warnings.filterwarnings("ignore")

# 📁 파일 경로
snp_path = "GWAS_SNP(3000).csv"
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

In [ ]:
# 📊 데이터 로딩
top_snp_df = pd.read_csv(snp_path)
geno_df = pd.read_csv(geno_path, index_col=0)
pheno_df = pd.read_excel(pheno_path, index_col=0)

# 📂 모델 저장 폴더 생성
os.makedirs("saved_models_rf", exist_ok=True)

In [ ]:
# 🎯 예측 대상 표현형 목록
traits = top_snp_df["Trait"].unique()
trait_name_map = {
    "과중": "과중 (g)",
    "과장": "과장 (mm)",
    "과폭": "과폭 (mm)",
    "과피두께": "과피두께 (mm)",
    "당도": "당도 (%)",
    "과실경도": "과실경도 (kg)"
}

results = []

for trait in traits:
    try:
        # SNP 목록 필터링
        top_snps = top_snp_df[top_snp_df["Trait"] == trait]["SNP"].tolist()
        top_snps = [snp for snp in top_snps if snp in geno_df.index]  # 존재하는 SNP만 사용
        if len(top_snps) == 0:
            print(f"❌ {trait} - 사용 가능한 SNP 없음")
            continue

        geno_top = geno_df.loc[top_snps].T  # 샘플 x SNP
        X = geno_top

        # 표현형 이름 매핑
        trait_col = trait_name_map.get(trait, trait)
        if trait_col not in pheno_df.columns:
            print(f"❌ {trait} → '{trait_col}' 컬럼이 표현형 데이터에 존재하지 않음")
            continue

        y = pheno_df[trait_col].loc[X.index]

        # 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 모델 학습
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # 모델 저장
        model_path = f"saved_models_rf/{trait}_rf_model.pkl"
        pd.to_pickle(model, model_path)

        # 성능 평가
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results.append({"표현형": trait, "MSE": mse, "R²": r2})

    except Exception as e:
        print(f"⚠️ {trait} 예측 중 오류 발생: {e}")

In [ ]:
# 결과 출력
results_df = pd.DataFrame(results)
print("모델 성능 요약:")
display(results_df)


모델 성능 요약:


,표현형,MSE,R²
0,과중,1001.943709,0.591109
1,과장,58.100354,0.380168
2,과폭,116.829500,0.532366
3,과피두께,0.632223,0.553022
4,과실경도,0.008832,0.296958
5,당도,0.507955,0.694395


In [ ]:
results_df.to_csv("RandomForest_모델성능.csv", index=False)

In [ ]:
# ✅ 앞쪽 10개 샘플 ID 선택
sample_ids = geno_df.columns[:10]

# ✅ 해당 샘플만 추출 (샘플 x SNP 구조로 전치)
new_samples_df = geno_df[sample_ids].T

# ✅ 저장
new_samples_df.to_csv("새로운_유전형.csv")


In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestRegressor

# 🔹 새로운 유전형 데이터 (샘플 x SNP)
new_samples_df = pd.read_csv("새로운_유전형.csv", index_col=0)

# 🔹 모델 경로
model_dir = "saved_models_rf"
all_predictions = []

for file in os.listdir(model_dir):
    if file.endswith(".pkl"):
        trait = file.replace("_rf_model.pkl", "")
        model_path = os.path.join(model_dir, file)

        try:
            model = pd.read_pickle(model_path)
            required_snps = model.feature_names_in_  # sklearn 1.0 이상 버전

            # 예측
            input_df = new_samples_df[required_snps]
            preds = model.predict(input_df)

            for sample_id, pred in zip(input_df.index, preds):
                all_predictions.append({
                    "샘플": sample_id,
                    "표현형": trait,
                    "예측값": pred
                })
        except Exception as e:
            print(f"⚠️ {trait} 예측 오류: {e}")

In [ ]:
# 결과 정리 및 저장
results_pred_df = pd.DataFrame(all_predictions)
print("✅ 새로운 샘플 예측 완료:")
display(results_pred_df.head())

results_pred_df.to_csv("RandomForest_예측결과.csv", index=False)

✅ 새로운 샘플 예측 완료:


,샘플,표현형,예측값
0,TC1_175,과장,48.5599
1,TC1_187,과장,39.4128
2,TC1_001,과장,38.1485
3,TC1_009,과장,26.8713
4,TC1_016,과장,38.1491
